In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from rouge_score import rouge_scorer

# Load the trained model and tokenizer
model_path = "model_kaggle"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

# Setup device for PyTorch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate a summary for a single input
def generate_summary(text):
    encodings = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_tensors="pt",
        padding="max_length",
        truncation=True
    )
    inputs = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    max_new_tokens = 25  # Adjust the number of tokens to generate
    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_beams=5,
        early_stopping=True
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Review Text and Given Summary
review_text = "The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound, and feels comfortable to play. However, some users have reported issues with the tuning stability."
actual_summary = "Good for beginners but has tuning stability issues."

# Generate summary
generated_summary = generate_summary(review_text)

# Compute ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(actual_summary, generated_summary)

# Print results
print("Given Review Text:", review_text)
print("Given Summary:", actual_summary)
print("Generated Summary:", generated_summary)
print("ROUGE-1 Scores:", scores['rouge1'])
print("ROUGE-2 Scores:", scores['rouge2'])
print("ROUGE-L Scores:", scores['rougeL'])



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Given Review Text: The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound, and feels comfortable to play. However, some users have reported issues with the tuning stability.
Given Summary: Good for beginners but has tuning stability issues.
Generated Summary: The Fender CD-60S Dreadnought Acoustic Guitar is a great instrument for beginners. It has a solid construction, produces a rich sound, and feels comfortable to play. However, some users have reported issues with the tuning stability. i have tried several different tuners and this is by far the best tuner i have found for the price it is a
ROUGE-1 Scores: Score(precision=0.1, recall=0.75, fmeasure=0.17647058823529416)
ROUGE-2 Scores: Score(precision=0.03389830508474576, recall=0.2857142857142857, fmeasure=0.060606060606060594)
ROUGE-L Scores: Score(precision=0.08333333333333333, recall=0.625, fmeasure=0.14705882352941174)
